In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv',encoding = "ISO-8859-1")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.value_counts()

In [ ]:
df.info()

In [ ]:
df['Unnamed: 2'].value_counts()

In [ ]:
df['Unnamed: 2'].nunique()

In [ ]:
df['Unnamed: 3'].nunique()

In [ ]:
df.head()

In [ ]:
df['v1'] = df['v1'].replace({'ham' : 0 , 'spam' : 1})

In [ ]:
df['v1'].value_counts()

In [ ]:
#on va traiter la colonne ""v2""

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopwords.words("english")

In [ ]:
df['len']=df['v2'].apply(len)

In [ ]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(10,5))
sns.distplot(df['len'],kde=False,color='red',hist=True)
plt.xlabel("Message Length",size=15)
plt.ylabel("Frequency",size=15)
plt.title("Length Histogram",size=15)

In [ ]:
from nltk.stem.porter import PorterStemmer
import re

In [ ]:
ps = PorterStemmer() #pour "text preprocessing"

message = []

for i in range(0, df.shape[0]):
    #accepter que les mots alphabétiques
    review = re.sub('[^a-zA-Z]', ' ', df['v2'][i])
    #convertir tous minuscule
    review = review.lower()
    #splitter chaque ligne
    review = review.split()
    #
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    #construire de nouveau la ligne
    review = ' '.join(review)
    message.append(review)

In [ ]:
df['clean_msg']=np.empty((len(message),1))
for i in range(len(message)):
    df['clean_msg'][i]=message[i]
df['clean_msg_len']=df['clean_msg'].apply(len)
df.head()

In [ ]:
df['v2'].describe()

In [ ]:
df['clean_msg'].describe()

In [ ]:
df.rename(columns={'v1':'label' , 'v2' : 'Msg'},inplace=True)

In [ ]:
df.head()

In [ ]:
X=df['clean_msg']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(message).toarray()

In [ ]:
Y=np.array(df['label'])

In [ ]:
#training model

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
#modéle1

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=100,criterion='gini',random_state=42)

In [ ]:
rfc.fit(x_train,y_train)

In [ ]:
y_pred = rfc.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
#c'est trés bien , mais on va aussi essayer avec d'autres méthodes

In [ ]:
#modéle2

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbk = GradientBoostingClassifier(random_state=42, n_estimators=100,min_samples_split=100)
gbk.fit(x_train, y_train)
gbk_predict = gbk.predict(x_test)
print("Gradient Boosting Score :",accuracy_score(y_test,gbk_predict ))

In [ ]:
#modéle3

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
mx=-1
for i in range(1,25):
    
    neigh=KNeighborsClassifier(n_neighbors = i).fit(x_train,y_train)
    yhat = neigh.predict(x_test)
    KNN_score=accuracy_score(y_test, yhat)
    print("KNN Accuracy at {} is {}".format(i,KNN_score))
    mx=max(mx,KNN_score)
    print("\n")
mx